In [2]:
import elasticsearch
import pandas as pd
import numpy as np
import pydicom
from datetime import datetime, timedelta

import os
import json

In [8]:
def get_all_dicoms(rootdir, index="id", onlydirs=True):
    id  = 0;
    for subdir, dirs, files in os.walk(rootdir):
        if 'newarrivals' in subdir or 'newarrivals' in dirs:
            continue
        if not onlydirs:
            for file in files:
                filepath = subdir + os.sep + file

                if filepath.endswith(".dcm"):
                    dc =  dicom_to_json(filepath)
                    es.index(index=index, doc_type='dicom', body=dc)
        else:
            if len(files) < 0 :
                continue
            thisdirflag = False
            for file in files:
                filepath = subdir + os.sep + file

                if filepath.endswith(".dcm"):
                    dc =  dicom_to_json(filepath)
                    dc["dir"] = subdir
                    dc["nSlices"] = len(files)
                    es.index(index=index, doc_type='dicom', body=dc)
                    thisdirflag = True
                if thisdirflag:
                    break
            
                

In [4]:
get_all_dicoms(r"z:/Fetal/New-Fetal-Org/")

OSError: [Errno 22] Invalid argument

In [8]:
dc = pydicom.read_file(dc_filename)

In [11]:
dc.

<generator object Dataset.elements at 0x000001B61E70E0F8>

In [108]:
a2 = a1[1]
type(a2.value)

pydicom.multival.MultiValue

In [73]:
dc_filename = r"Z:/Fetal/Siemens_new/AKKERMAN_YULIA\Study20181011_122923_791000\AKYU_Se01_localizer\MR001001006.dcm"
dc = dicom_to_json(dc_filename)
dc

InstanceCreationDate = 2018-10-11 00:00:00  *** 
12:30:05  +  2018-10-11 00:00:00
 =  2018-10-11 12:30:05
StudyDate = 2018-10-11 00:00:00  *** 
SeriesDate = 2018-10-11 00:00:00  *** 
AcquisitionDate = 2018-10-11 00:00:00  *** 
ContentDate = 2018-10-11 00:00:00  *** 
12:29:23  +  2018-10-11 00:00:00
 =  2018-10-11 12:29:23
12:29:59  +  2018-10-11 00:00:00
 =  2018-10-11 12:29:59
12:29:59  +  2018-10-11 00:00:00
 =  2018-10-11 12:29:59
12:30:05  +  2018-10-11 00:00:00
 =  2018-10-11 12:30:05
PatientBirthDate = 1985-05-14 00:00:00  *** 
PerformedProcedureStepStartDate = 2018-10-11 00:00:00  *** 
12:29:23  +  2018-10-11 00:00:00
 =  2018-10-11 12:29:23


{'SpecificCharacterSet': 'ISO_IR 100',
 'InstanceCreationDate': datetime.datetime(2018, 10, 11, 12, 30, 5),
 'SOPClassUID': '1.2.840.10008.5.1.4.1.1.4',
 'SOPInstanceUID': '1.3.12.2.1107.5.2.19.45773.2018101112300420758211',
 'StudyDate': datetime.datetime(2018, 10, 11, 12, 29, 23),
 'SeriesDate': datetime.datetime(2018, 10, 11, 12, 29, 59),
 'AcquisitionDate': datetime.datetime(2018, 10, 11, 12, 29, 59),
 'ContentDate': datetime.datetime(2018, 10, 11, 12, 30, 5),
 'AccessionNumber': 3400276021864886.0,
 'Modality': 'MR',
 'Manufacturer': 'SIEMENS',
 'InstitutionName': 'TEL AVIV MEDICAL CENTER',
 'InstitutionAddress': 'Street StreetNo,Tel-Aviv,District,IL,ZIP',
 'ReferringPhysicianName': '',
 'StationName': 'AWP45773',
 'StudyDescription': 'ICHILOV^Fetus',
 'CodeValue': 'KE10921',
 'CodingSchemeDesignator': 'CCG_CSTemp',
 'CodeMeaning': 'MRI FETAL',
 'SeriesDescription': 'localizer',
 'InstitutionalDepartmentName': 'Department',
 'PhysiciansOfRecord': '1^1^^^',
 'PerformingPhysicianNam

In [4]:
def dicom_to_json(dc_filename):
    dc = pydicom.read_file(dc_filename)
    json_dict = {}
    for elem in dc.iterall():
        if (len(elem.keyword.strip()) == 0) or (elem.tag.group == 0x7fe0):
            continue
        elif isinstance(elem.value, pydicom.multival.MultiValue):
            continue
        elif 'Date' in elem.keyword:
            try:
                dateobj = datetime.strptime(str(elem.value), "%Y%m%d")
            except:
                continue
            if elem.keyword in json_dict.keys():
#                 print(elem.keyword, dateobj, " += ", json_dict[elem.keyword])
                tt = json_dict[elem.keyword]
                dateobj += timedelta(hours=tt.hour, minutes=tt.minute, seconds=tt.second)
#             print(elem.keyword, "=", dateobj, " *** ")
            json_dict[elem.keyword]=dateobj
        elif 'Time' in elem.keyword:
            elemname = elem.keyword.replace('Time', 'Date')
            if not hasattr(dc,elemname):
                json_dict[elem.keyword]=str(elem.value)
                continue
            
            elemval = str(elem.value).split(".")[0]
            try:
                tt = datetime.strptime(elemval, "%H%M%S")
                dateobj = timedelta(hours=tt.hour, minutes=tt.minute, seconds=tt.second)
            except:
                 continue
            
            if elemname in json_dict:
#                 print (dateobj, " + ", json_dict[elemname])
                dateobj += json_dict[elemname]
                json_dict[elemname]=dateobj
#                 print (" = ", dateobj)
            else:
#                 print (elemname, " = " , dateobj)
                json_dict[elemname]=dateobj
                
        elif elem.keyword in ['ChemicalShiftReference', 'TagSpacingFirstDimension', 'TagAngleFirstAxis', 'FillerOrderNumberImagingServiceRequest', 'CodingSchemeVersion', 'AccessionNumber', 'PerformedLocation', 'PerformedStationName','StudyID','ScheduledProcedureStepID', 'PerformedProcedureStepID','DeviceSerialNumber']:
            json_dict[elem.keyword]=str(elem.value)           
        else:
#             try:
#                 json_dict[elem.keyword]=float(elem.value)
#             except:
            json_dict[elem.keyword]=str(elem.value)
    return json_dict

In [13]:
def connect_to_es():
    es = elasticsearch.Elasticsearch([{'host': '10.101.85.13', 'port':9200}])
    return es

In [14]:
es = connect_to_es()

In [ ]:
es.index(index="id2", doc_type="talk2", body={"Author": "Dafna", "StudyID":"3400279534221967", "Data": "Bla bla bla 2"})

In [16]:
get_all_dicoms(r"/media/df3/Fetal/Data-All/", index="tasmc_fetal", onlydirs=True)

In [38]:
get_all_dicoms(r"//fmri-df1/users/", index="all-df10", onlydirs=True)

InvalidDicomError: File is missing DICOM File Meta Information header or the 'DICM' prefix is missing from the header. Use force=True to force reading.

In [5]:
def get_all_rda(rootdir, index="id", onlydirs=True):
    id  = 0;
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            filepath = subdir + os.sep + file

            if filepath.endswith(".rda"):
                print(filepath)

In [6]:
get_all_rda(r"Z:/Fetal/New-Fetal-Org/")

Z:/Fetal/New-Fetal-Org/new\YAISH_NESIA_NOA\mrs\MrSpec.20181104.131430-Yaish Nesia Noa.rda
Z:/Fetal/New-Fetal-Org/new\NAVON_HILA\MRS\MrSpec.20181129.094501 Navon_Hila.rda
Z:/Fetal/New-Fetal-Org/new\AVIVI_ALEKSANDRA\MRS\MrSpec.20181014.092823-AVIV ALEKS.rda
Z:/Fetal/New-Fetal-Org/new\GERTLER_MEIRAV_MARINA-v\mrs\MrSpec.20181108.150020-Gartler Meirav_Marina.rda
Z:/Fetal/New-Fetal-Org/new\HIRSCHHORN_LIAT_BLUMA-v\mrs\MrSpec.20181213.093238 hischhoren liat 2.rda
Z:/Fetal/New-Fetal-Org/new\HIRSCHHORN_LIAT_BLUMA-v\mrs\MrSpec.20181213.092801 hirschhoren liat.rda
Z:/Fetal/New-Fetal-Org/new\MASHIAH_MORAN\MRS\MrSpec.20181004.153951-Mashiah Moran.rda
Z:/Fetal/New-Fetal-Org/new\ALBO_IRINA-v\mrs\MrSpec.20181104.131746-Albo Irina.rda
Z:/Fetal/New-Fetal-Org/new\TURNER_RUTH\MRS\MrSpec.20181004.103651-TurnerRuth.rda
Z:/Fetal/New-Fetal-Org/new\HOICHRACH_AVIGAIL-v\MRS\MrSpec.20181004.153759-Hoichrach Avigail.rda
Z:/Fetal/New-Fetal-Org/new\Zaks Eva\mrs\MrSpec.20181108.150158-Zaks Eva.rda
Z:/Fetal/New-Fetal-O